In [ ]:
#!pip install aitextgen
#!pip install sentence_transformers
#!pip install fuzzywuzzy

In [ ]:
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive, copy_file_to_gdrive
from aitextgen import aitextgen
import pandas as pd
from collections import Counter

In [ ]:
from sentence_transformers import SentenceTransformer
# nltk.download('wordnet')
# nltk.download('stopwords')
from sklearn.cluster import KMeans
import pandas as pd 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import re

In [ ]:
class Clusters():
    def __init__(self, n=10):
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.num_clusters = n

    def train(self, txt):
        corpus_embeddings = self.embedder.encode(txt.values)
        self.kmeans = KMeans(n_clusters=self.num_clusters,init='k-means++',max_iter=300,n_init=10,random_state=0)
        self.kmeans.fit(corpus_embeddings)
        return self.kmeans.predict(corpus_embeddings)


    def test(self, txt):
        corpus_embeddings = self.embedder.encode(txt.values)
        return self.kmeans.predict(corpus_embeddings)

In [ ]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
train = pd.read_json('train.json')
train['text'] = "Category: " +  train.category + " # " + train.Rationale.str.replace('"', '') + ' # ' + train.Problem
train['prompt'] = "Category: " +  train.category + " # " + train.Rationale.str.replace('"', '') + ' # '
train['clean'] = train[['text']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False,lst_stopwords=nltk.corpus.stopwords.words("english")))

train.text.to_csv('rationale_train.csv', index=False, header=None)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
ncluster = 6
physics = Clusters(ncluster)
physics_train = physics.train(train[train.category == 'physics']['clean'])
physics_test = physics.test(train[train.category == 'physics']['clean'])

In [ ]:
phys_expl = train[train.category == 'physics']
phys_expl['cluster'] = physics_test
phys_expl['prompt'] = "Category: " + phys_expl.category + " # " + phys_expl.Rationale.str.replace('"', '') + " # "

In [ ]:
phys_expl

In [ ]:
phys_expl['generated'] = phys_expl.apply(lambda row: ai_0.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_expl['generated']

In [ ]:
texts = []
for i in range(ncluster):
  texts.append(' '.join(phys_expl[phys_expl.cluster == i].clean.tolist()))

In [ ]:
"""
ai = aitextgen()
ai.train('rationale_train_0.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1,
         output_dir='trained_model_0'
         )
"""

In [ ]:
"""
ai = aitextgen()
ai.train('rationale_train.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )
"""

In [ ]:
"""
ai_5 = aitextgen()
ai_5.train('rationale_train_5.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1,
         output_dir='trained_model_5'
         )
"""

In [ ]:
"""
ai_3 = aitextgen()
ai_3.train('rationale_train_3.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1,
         output_dir='trained_model_3'
         )
"""

In [ ]:
"""
ai_4 = aitextgen()
ai_4.train('rationale_train_4.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1,
         output_dir='trained_model_4'
         )
"""

In [ ]:
test = pd.read_json('test.json')
test['prompt'] = "Category: " + test.category + " # " + test.Rationale.str.replace('"', '') + " # "
test['text'] = "Category: " + test.category + " # " + test.Rationale.str.replace('"', '') + " # " + test.Problem
test

In [ ]:
phys_test = test[test.category == 'physics']
phys_test['clean'] = phys_test[['text']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False,lst_stopwords=nltk.corpus.stopwords.words("english")))
phys_test['cluster'] = physics.test(phys_test)
phys_test_0 = phys_test[phys_test.cluster == 0]

In [ ]:
#ai = aitextgen(model_folder='trained_model', to_gpu=True)
ai_0 = aitextgen(model_folder='trained_model_0', to_gpu=True)
#ai_1 = aitextgen(model_folder='trained_model_1', to_gpu=True)

In [ ]:
phys_test_0['generated'] = phys_test_0.apply(lambda row: ai_0.generate(n=5, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_0['generated']

In [ ]:
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

class Comparer():
    def __init__(self, l=1.0):
        self.l = l
        self.pattern = r'[0-9][0-9]*'

    def convert(self, s):
        nums = list(filter(lambda elem: elem.isnumeric(), s.split(' ')))
        new_string = re.sub(self.pattern, 'Var', s)
        return new_string, nums

    def lossy_numbers(self, l1, l2):
        loss = 0
        for i in range(min(len(l1), len(l2))):
            if l1[i] != l2[i]:
                loss += 1
        loss += max(len(l1), len(l2)) - min(len(l1), len(l2))
        return loss


    def loss_sentences(self, s1, s2):
        m1, l1 = self.convert(s1)
        m2, l2 = self.convert(s2)
        return fuzz.partial_ratio(m1,m2)/100.0, self.l*(self.lossy_numbers(l1, l2))


In [ ]:
c = Comparer()

In [ ]:
phys_test_0['score'] = phys_test_0.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_0

In [ ]:
phys_test_0['gpt_generated'] = phys_test_0.apply(lambda row: ai.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_0['gpt_generated']

In [ ]:
phys_test_0['gpt_score'] = phys_test_0.apply(lambda row: c.loss_sentences(row['text'], row['gpt_generated'][0]), axis=1)
phys_test_0

In [ ]:
phys_test_0.to_csv('output_0.csv')

In [ ]:
phys_test_0 = pd.read_csv('output_0.csv').iloc[:, 1:]
phys_test_0['score'] = phys_test_0.apply(lambda row: eval(row['score']), axis=1)
phys_test_0['gpt_score'] = phys_test_0.apply(lambda row: eval(row['gpt_score']), axis=1)
phys_test_0

In [ ]:
phys_test_0[['s_score', 'n_score']] = phys_test_0.score.tolist()
phys_test_0[['s_gptscore', 'n_gptscore']] = phys_test_0.gpt_score.tolist()
print(phys_test_0.s_score.mean())
print(phys_test_0.s_gptscore.mean())
print(phys_test_0.n_score.mean())
print(phys_test_0.n_gptscore.mean())

In [ ]:
phys_test_0.to_csv('output_0.csv')

In [ ]:
phys_test['gpt_generated'] = phys_test.apply(lambda row: ai.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test['gpt_score'] = phys_test.apply(lambda row: c.loss_sentences(row['text'], row['gpt_generated'][0]), axis=1)

In [ ]:
phys_test.to_csv('physicsgenerated.csv')

In [ ]:
phys_test = pd.read_csv('physicsgenerated.csv')
phys_test['gpt_score'] = phys_test.apply(lambda row: eval(row['gpt_score']), axis=1)

In [ ]:
phys_test[['s_gptscore', 'n_gptscore']] = phys_test['gpt_score'].tolist()
print(phys_test.s_gptscore.mean())
print(phys_test.n_gptscore.mean())

In [ ]:
phys_test_1 = phys_test[phys_test.cluster == 1]
phys_test_1['gpt_score'] = phys_test_1.apply(lambda row: c.loss_sentences(row['text'], row['gpt_generated'][0]), axis=1)
phys_test[['s_gptscore', 'n_gptscore']] = phys_test['gpt_score'].tolist()

In [ ]:
phys_test_1['generated'] = phys_test_1.apply(lambda row: ai_1.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_1['score'] = phys_test_1.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)

In [ ]:
phys_test_1[['s_score', 'n_score']] = phys_test_1.score.tolist()
phys_test_1.to_csv('output_1.csv')
print(phys_test_1.s_score.mean())
print(phys_test_1.n_score.mean())
print(phys_test[phys_test.cluster == 1].s_gptscore.mean())
print(phys_test[phys_test.cluster == 1].n_gptscore.mean())

In [ ]:
ai_2 = aitextgen(model_folder='trained_model_2', to_gpu=True)
ai_3 = aitextgen(model_folder='trained_model_3', to_gpu=True)
ai_4 = aitextgen(model_folder='trained_model_4', to_gpu=True)
ai_5 = aitextgen(model_folder='trained_model_5', to_gpu=True)

In [ ]:
phys_test_2 = phys_test[phys_test.cluster == 2]
phys_test_2['generated'] = phys_test_2.apply(lambda row: ai_2.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_2['score'] = phys_test_2.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_2[['s_score', 'n_score']] = phys_test_2.score.tolist()
phys_test_2.to_csv('output_2.csv')
print(phys_test_2.s_score.mean())
print(phys_test_2.n_score.mean())

In [ ]:
phys_test_3 = phys_test[phys_test.cluster == 3]
phys_test_3['generated'] = phys_test_3.apply(lambda row: ai_3.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_3['score'] = phys_test_3.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_3[['s_score', 'n_score']] = phys_test_3.score.tolist()
phys_test_3.to_csv('output_3.csv')
print(phys_test_3.s_score.mean())
print(phys_test_3.n_score.mean())

In [ ]:
phys_test_4 = phys_test[phys_test.cluster == 4]
phys_test_4['generated'] = phys_test_4.apply(lambda row: ai_4.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_4['score'] = phys_test_4.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_4[['s_score', 'n_score']] = phys_test_4.score.tolist()
phys_test_4.to_csv('output_4.csv')
print(phys_test_4.s_score.mean())
print(phys_test_4.n_score.mean())

In [ ]:
phys_test_5 = phys_test[phys_test.cluster == 5]
phys_test_5['generated'] = phys_test_5.apply(lambda row: ai_5.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_5['score'] = phys_test_5.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_5[['s_score', 'n_score']] = phys_test_5.score.tolist()
phys_test_5.to_csv('output_5.csv')
print(phys_test_5.s_score.mean())
print(phys_test_5.n_score.mean())

In [ ]:
(phys_test_0.s_score.mean() + phys_test_1.s_score.mean() + phys_test_2.s_score.mean() + phys_test_3.s_score.mean() + \
 phys_test_4.s_score.mean() + phys_test_5.s_score.mean()) / 6

In [ ]:
(phys_test_0.n_score.mean() + phys_test_1.n_score.mean() + phys_test_2.n_score.mean() + phys_test_3.n_score.mean() + \
 phys_test_4.n_score.mean() + phys_test_5.n_score.mean()) / 6

In [ ]:
from sentence_transformers import SentenceTransformer
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd 
import re
import nltk
import torch
import numpy as np
import tensorflow as tf
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

class Comparer():
    def __init__(self, l=1.0):
        self.l = l
        self.pattern = r'[0-9][0-9]*'

    def convert(self, s):
        nums = list(filter(lambda elem: elem.isnumeric(), s.split(' ')))
        new_string = re.sub(self.pattern, 'Var', s)
        return new_string, nums

    def lossy_numbers(self, l1, l2):
        loss = 0
        for i in range(min(len(l1), len(l2))):
            if l1[i] != l2[i]:
                loss += 1
        loss += max(len(l1), len(l2)) - min(len(l1), len(l2))
        return loss


    def loss_sentences(self, s1, s2):
        m1, l1 = self.convert(s1)
        m2, l2 = self.convert(s2)
        #print(fuzz.ratio(m1,m2))
        return -fuzz.partial_ratio(m1,m2)/100.0 + self.l*(self.lossy_numbers(l1, l2))
    
    def loss_sentences_output(self, s1, s2):
        m1, l1 = self.convert(s1)
        m2, l2 = self.convert(s2)
        return -fuzz.partial_ratio(m1,m2)/100.0, self.l*(self.lossy_numbers(l1, l2))

def get_val(s):
    return list(filter(lambda elem: elem.isnumeric(), s.split(' ')))

def get_label(s, l):
    return list(map(lambda elem: 2*int(elem in l)-1, s.split(' ')))

def generate_numbers(data, c):
    data['labels'] = data.apply(lambda r: c.loss_sentences(r['clean'], r['generated_clean']) , axis=1)
    #data['labels'] = np.log(data['labels'])
    data['labels'] = data['labels']/data['labels'].max()
    return data['labels']

In [ ]:
output = pd.read_csv('physicsgenerated.csv')
output_0 = pd.read_csv('output_0.csv')
output_1 = pd.read_csv('output_1.csv')
output_2 = pd.read_csv('output_2.csv')
output_3 = pd.read_csv('output_3.csv')
output_4 = pd.read_csv('output_4.csv')
output_5 = pd.read_csv('output_5.csv')

In [ ]:
# Build a model
inputs = layers.Input(shape=(384,))
layer1 = layers.Dense(128, activation='relu')(inputs)
layer2 = layers.Dense(128)(layer1)
layer3 = keras.layers.LeakyReLU(alpha=0.3)(layer2)
predictions = layers.Dense(1, activation='sigmoid')(layer3)
model = keras.Model(inputs=inputs, outputs=predictions)
print('Model Built')
# Define custom loss

In [ ]:
output_0['generated']

In [ ]:
output_0['generated_clean'] = output_0[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean']

In [ ]:
output_0['generated_clean'] = output_0[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_0, Comparer())
output_0.to_csv('output_0.csv')

output_1['generated_clean'] = output_1[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_1, Comparer())
output_1.to_csv('output_1.csv')

output_2['generated_clean'] = output_2[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_2, Comparer())
output_2.to_csv('output_2.csv')

output_3['generated_clean'] = output_3[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_3, Comparer())
output_3.to_csv('output_3.csv')

output_4['generated_clean'] = output_4[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_4, Comparer())
output_4['loss'] = labels
output_4.to_csv('output_4.csv')

output_5['generated_clean'] = output_5[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_5, Comparer())
output_5.to_csv('output_5.csv')

output['generated_clean'] = output[['gpt_generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output, Comparer())
output.to_csv('output.csv')

In [ ]:
output_0['generated_clean'] = output_0[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_0, Comparer())
output_0.to_csv('output_0.csv')

# Compile the model
model.compile(optimizer='adam',
              loss='mse', # Call the loss function with the selected layer
              metrics=['accuracy'])
print('Model  compiled')
embedder = SentenceTransformer('all-MiniLM-L6-v2')
model.fit(embedder.encode(output_0['generated_clean'].values), np.array(labels))

In [ ]:
import keras

In [ ]:
output_0 = pd.read_csv('output_0.csv')

In [ ]:
output_0['generated'] = output_0.apply(lambda row: ai_0.generate(n=5, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
output_0[['generated_0', 'generated_1', 'generated_2', 'generated_3', 'generated_4']] = \
    output_0['generated'].tolist()

In [ ]:
output_0['generated_clean_0'] = output_0[['generated_0']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean_1'] = output_0[['generated_1']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean_2'] = output_0[['generated_2']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean_3'] = output_0[['generated_3']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean_4'] = output_0[['generated_4']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))

In [ ]:
output_0.to_csv('output_0.csv')

In [ ]:
output_0['generated_clean_0'][0]

In [ ]:
model = keras.models.load_model('.')
embedder = SentenceTransformer('all-MiniLM-L6-v2')

min_score = 10000
min_col = ''
for col in ['generated_clean_0', 'generated_clean_1', 'generated_clean_2', 'generated_clean_3', 'generated_clean_4']:
    col_name = col + '_score'
    output_0[col_name] = model.predict(embedder.encode(output_0[col].values))
cols = ['generated_clean_0_score', 'generated_clean_1_score', 'generated_clean_2_score', 'generated_clean_3_score', 'generated_clean_4_score']
output_0['min_col'] = output_0[cols].idxmin(axis=1)
output_0['min_col'] = output_0.apply(lambda x: x['min_col'].strip('_score'), axis=1)
output_0['output'] = output_0.apply(lambda x: x[x['min_col']], axis=1)
output_0['output']

In [ ]:
output_0.to_csv('output_0.csv')

In [ ]:
c = Comparer()

In [ ]:
output_0['score'] = output_0.apply(lambda row: c.loss_sentences_output(row['clean'], row['output']), axis=1)
output_0[['s_score', 'n_score']] = output_0.score.tolist()
output_0.to_csv('output_0.csv')

In [ ]:
s1 = "a rectangular floor that measures 15 meters by 18 meters is to be covered with carpet squares that each measure 10 meters by 3 meters. if the carpet squares cost $ 12 apiece, how much will the carpet squares cost?"
s2 = "a rectangular floor that measures 15 meters by 18 meters is to be covered with carpet squares that each measure 3 meters by 3 meters . if the carpet squares cost $12 apiece , what is the total cost for the number of carpet squares needed to cover the floor ?"
c.loss_sentences(s1,s2)